# Lab 1: LLM Deployment & Empirical Analysis

This notebook analyzes the results of deploying three different LLMs (TinyLlama, Qwen, Gemma) using Ollama. We compare their performance on three tasks (Generation, Classification, Summarization) under two configurations:

1.  **Basic**: Default Ollama parameters.
2.  **Tuned**: Custom parameters (`temperature=0.5`, `top_p=0.8`, `max_tokens=100`).

In [11]:
import json
import pandas as pd
from pathlib import Path

RESULTS_PATH = Path("results.jsonl")

def load_results():
    data = []
    if not RESULTS_PATH.exists():
        print("Results file not found!")
        return pd.DataFrame()
        
    with open(RESULTS_PATH, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip():
                data.append(json.loads(line))
    return pd.DataFrame(data)

df = load_results()
if not df.empty:
    df["response_length"] = df["response"].apply(len)
    print(f"Loaded {len(df)} runs.")
else:
    print("No data loaded.")

Loaded 18 runs.


## 1. Performance Metrics

We aggregate the results by **Model** and **Mode** to see the impact of tuning on latency (`duration_s`) and verbosity (`response_length`).

In [12]:
if not df.empty:
    summary = df.groupby(["model", "mode"]).agg({
        "duration_s": ["mean", "std"],
        "response_length": ["mean", "std"]
    })
    display(summary)

duration_s            response_length             
                      mean        std            mean          std
model     mode                                                    
Gemma     basic  13.608555  18.500920      975.333333  1552.297115
          tuned   9.212318  10.812523      999.333333  1569.440134
Qwen      basic   3.540214   1.912451      668.000000   993.995473
          tuned   2.512976   0.224832      301.000000   276.570063
TinyLlama basic   4.055043   2.724723     1177.000000  1764.263869
          tuned   4.610591   3.536320     1580.000000  2457.992067

## 2. Impact of Tuning

### Explanation of Parameters
- **Temperature (0.5 vs Default)**: Lower temperature makes the model more deterministic and focused. We expect "Tuned" responses to be less random.
- **Top_P (0.8)**: Restricts the token pool to the top 80% probability mass, further reducing tail-end randomness.
- **Max Tokens (100)**: Limits the generation length. This significantly reduces latency for verbose models but might cut off answers.

### Observation
Compare the **Basic** vs **Tuned** rows in the table above. You should generally see:
- **Lower Duration** in Tuned mode (due to `max_tokens` limit).
- **Lower Response Length** in Tuned mode (due to `max_tokens` limit).
- **Lower Variance (std)** in Tuned mode (due to lower temperature).

## 3. Qualitative Analysis: Sample Outputs

Let's look at the actual responses for a specific prompt to see quality differences.

In [13]:
pd.set_option('display.max_colwidth', None)

def show_samples(task_type):
    if df.empty: return
    subset = df[df["type"] == task_type][["model", "mode", "prompt", "response"]]
    display(subset.sort_values(by=["model", "mode"]))

# Change this to 'generation', 'classification', or 'summarization'
show_samples("generation")

model   mode  \
12      Gemma  basic   
13      Gemma  tuned   
6        Qwen  basic   
7        Qwen  tuned   
0   TinyLlama  basic   
1   TinyLlama  tuned   

                                                                         prompt  \
12  Write a short story about a robot learning to paint. Output only the story.   
13  Write a short story about a robot learning to paint. Output only the story.   
6   Write a short story about a robot learning to paint. Output only the story.   
7   Write a short story about a robot learning to paint. Output only the story.   
0   Write a short story about a robot learning to paint. Output only the story.   
1   Write a short story about a robot learning to paint. Output only the story.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [14]:
show_samples("classification")

,model,mode,prompt,response
14,Gemma,basic,"Classify the following text into one of these categories: [Sports, Politics, Technology]. Text: 'The new processor offers 20% better performance.' Output ONLY the category name.",Technology\n
15,Gemma,tuned,"Classify the following text into one of these categories: [Sports, Politics, Technology]. Text: 'The new processor offers 20% better performance.' Output ONLY the category name.",Technology\n
8,Qwen,basic,"Classify the following text into one of these categories: [Sports, Politics, Technology]. Text: 'The new processor offers 20% better performance.' Output ONLY the category name.",Sports
9,Qwen,tuned,"Classify the following text into one of these categories: [Sports, Politics, Technology]. Text: 'The new processor offers 20% better performance.' Output ONLY the category name.",Sports.
2,TinyLlama,basic,"Classify the following text into one of these categories: [Sports, Politics, Technology]. Text: 'The new processor offers 20% better performance.' Output ONLY the category name.",Sportspoliticstechnology
3,TinyLlama,tuned,"Classify the following text into one of these categories: [Sports, Politics, Technology]. Text: 'The new processor offers 20% better performance.' Output ONLY the category name.",Sports: This is a Sports category entry only.


In [15]:
show_samples("summarization")

,model,mode,prompt,response
16,Gemma,basic,"Summarize the following text in one sentence: 'Photosynthesis is a process used by plants and other organisms to convert light energy into chemical energy that, through cellular respiration, can later be released to fuel the organism's activities. This chemical energy is stored in carbohydrate molecules, such as sugars and starches, which are synthesized from carbon dioxide and water.' Output ONLY the summary.","Photosynthesis converts light energy into chemical energy stored in carbohydrates, which are then utilized by organisms through cellular respiration."
17,Gemma,tuned,"Summarize the following text in one sentence: 'Photosynthesis is a process used by plants and other organisms to convert light energy into chemical energy that, through cellular respiration, can later be released to fuel the organism's activities. This chemical energy is stored in carbohydrate molecules, such as sugars and starches, which are synthesized from carbon dioxide and water.' Output ONLY the summary.","Photosynthesis transforms light energy into chemical energy stored in carbohydrates, utilizing carbon dioxide and water, which organisms then access through cellular respiration."
10,Qwen,basic,"Summarize the following text in one sentence: 'Photosynthesis is a process used by plants and other organisms to convert light energy into chemical energy that, through cellular respiration, can later be released to fuel the organism's activities. This chemical energy is stored in carbohydrate molecules, such as sugars and starches, which are synthesized from carbon dioxide and water.' Output ONLY the summary.","Photosynthesis is a process used by plants and other organisms to convert light energy into chemical energy that, through cellular respiration, can later be released to fuel the organism'"
11,Qwen,tuned,"Summarize the following text in one sentence: 'Photosynthesis is a process used by plants and other organisms to convert light energy into chemical energy that, through cellular respiration, can later be released to fuel the organism's activities. This chemical energy is stored in carbohydrate molecules, such as sugars and starches, which are synthesized from carbon dioxide and water.' Output ONLY the summary.","Photosynthesis is a process used by plants and other organisms to convert light energy into chemical energy that, through cellular respiration, can later be released to fuel the organism's activities. This chemical energy is stored in carbohydrate molecules, such as sugars and starches, which are synthesized from carbon dioxide and water."
4,TinyLlama,basic,"Summarize the following text in one sentence: 'Photosynthesis is a process used by plants and other organisms to convert light energy into chemical energy that, through cellular respiration, can later be released to fuel the organism's activities. This chemical energy is stored in carbohydrate molecules, such as sugars and starches, which are synthesized from carbon dioxide and water.' Output ONLY the summary.","Photosynthesis is a process used by organisms to convert light into chemical energy that can be stored in cellular reserves. This energy is eventually released through cellular respiration, which converts sugars and starches synthesized from carbon dioxide and water back into other forms of energy."
5,TinyLlama,tuned,"Summarize the following text in one sentence: 'Photosynthesis is a process used by plants and other organisms to convert light energy into chemical energy that, through cellular respiration, can later be released to fuel the organism's activities. This chemical energy is stored in carbohydrate molecules, such as sugars and starches, which are synthesized from carbon dioxide and water.' Output ONLY the summary.",Photosynthesis converts light energy into chemical energy that can be stored in carbohydrate molecules by cellular respiration. This process occurs within plant and other organisms and is crucial for their survival